In [231]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

## Importing data into pandas dataframe

In [232]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [233]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [234]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Splitting the ticket info to two categories and normalizing the name

In [235]:
processed_train_df = pd.DataFrame()
processed_test_df = pd.DataFrame()

def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
processed_train_df = preprocess(train_df)
processed_test_df = preprocess(test_df)

In [236]:
processed_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [237]:
processed_test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,892,3,Kelly Mr James,male,34.5,0,0,330911,7.8292,NaN,Q,330911,NONE
1,893,3,Wilkes Mrs James Ellen Needs,female,47.0,1,0,363272,7.0000,NaN,S,363272,NONE
2,894,2,Myles Mr Thomas Francis,male,62.0,0,0,240276,9.6875,NaN,Q,240276,NONE
3,895,3,Wirz Mr Albert,male,27.0,0,0,315154,8.6625,NaN,S,315154,NONE
4,896,3,Hirvonen Mrs Alexander Helga E Lindqvist,female,22.0,1,1,3101298,12.2875,NaN,S,3101298,NONE


## Preparing the datasets

In [238]:
train_data = processed_train_df.drop(columns=["PassengerId", "Survived", "Ticket"], inplace=False)
train_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,3,Braund Mr Owen Harris,male,22.0,1,0,7.2500,NaN,S,21171,A/5
1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,71.2833,C85,C,17599,PC
2,3,Heikkinen Miss Laina,female,26.0,0,0,7.9250,NaN,S,3101282,STON/O2.
3,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,53.1000,C123,S,113803,NONE
4,3,Allen Mr William Henry,male,35.0,0,0,8.0500,NaN,S,373450,NONE


In [239]:
dummies = pd.get_dummies(train_data["Sex"])
train_data = pd.concat([train_data, dummies], axis=1)
train_data.drop(["Sex"], axis=1, inplace=True)
train_data.head()

,Pclass,Name,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_number,Ticket_item,female,male
0,3,Braund Mr Owen Harris,22.0,1,0,7.2500,NaN,S,21171,A/5,False,True
1,1,Cumings Mrs John Bradley Florence Briggs Thayer,38.0,1,0,71.2833,C85,C,17599,PC,True,False
2,3,Heikkinen Miss Laina,26.0,0,0,7.9250,NaN,S,3101282,STON/O2.,True,False
3,1,Futrelle Mrs Jacques Heath Lily May Peel,35.0,1,0,53.1000,C123,S,113803,NONE,True,False
4,3,Allen Mr William Henry,35.0,0,0,8.0500,NaN,S,373450,NONE,False,True


In [240]:
dummies = pd.get_dummies(train_data["Embarked"])
train_data = pd.concat([train_data, dummies], axis=1)
train_data.drop("Embarked", axis=1, inplace=True)
train_data.head()

,Pclass,Name,Age,SibSp,Parch,Fare,Cabin,Ticket_number,Ticket_item,female,male,C,Q,S
0,3,Braund Mr Owen Harris,22.0,1,0,7.2500,NaN,21171,A/5,False,True,False,False,True
1,1,Cumings Mrs John Bradley Florence Briggs Thayer,38.0,1,0,71.2833,C85,17599,PC,True,False,True,False,False
2,3,Heikkinen Miss Laina,26.0,0,0,7.9250,NaN,3101282,STON/O2.,True,False,False,False,True
3,1,Futrelle Mrs Jacques Heath Lily May Peel,35.0,1,0,53.1000,C123,113803,NONE,True,False,False,False,True
4,3,Allen Mr William Henry,35.0,0,0,8.0500,NaN,373450,NONE,False,True,False,False,True


In [241]:
train_data["male"] = train_data["male"].astype(int)
train_data["female"] = train_data["female"].astype(int)
train_data["C"] = train_data["C"].astype(int)
train_data["Q"] = train_data["Q"].astype(int)
train_data["S"] = train_data["S"].astype(int)


In [242]:
def count_cabins(cabin):
    if pd.isna(cabin):  
        return 0
    else:
        return cabin.count(' ') + 1  
    
train_data['num_cabins'] = train_data['Cabin'].apply(count_cabins)

In [243]:
def has_ticket_item(item):
    if item == "NONE":
        return 0
    else:
        return 1
    
train_data['has_ticket_item'] = train_data['Ticket_item'].apply(has_ticket_item)

In [244]:
train_data = train_data[["Name", "Pclass", "female", "male", "Age", "SibSp", "Parch", "Fare", "num_cabins", "C", "Q", "S", "Ticket_number", "has_ticket_item"]]
train_data.head()

,Name,Pclass,female,male,Age,SibSp,Parch,Fare,num_cabins,C,Q,S,Ticket_number,has_ticket_item
0,Braund Mr Owen Harris,3,0,1,22.0,1,0,7.2500,0,0,0,1,21171,1
1,Cumings Mrs John Bradley Florence Briggs Thayer,1,1,0,38.0,1,0,71.2833,1,1,0,0,17599,1
2,Heikkinen Miss Laina,3,1,0,26.0,0,0,7.9250,0,0,0,1,3101282,1
3,Futrelle Mrs Jacques Heath Lily May Peel,1,1,0,35.0,1,0,53.1000,1,0,0,1,113803,0
4,Allen Mr William Henry,3,0,1,35.0,0,0,8.0500,0,0,0,1,373450,0


In [245]:
test_data = processed_test_df.drop(columns=["PassengerId", "Ticket"], inplace= False)
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,3,Kelly Mr James,male,34.5,0,0,7.8292,NaN,Q,330911,NONE
1,3,Wilkes Mrs James Ellen Needs,female,47.0,1,0,7.0000,NaN,S,363272,NONE
2,2,Myles Mr Thomas Francis,male,62.0,0,0,9.6875,NaN,Q,240276,NONE
3,3,Wirz Mr Albert,male,27.0,0,0,8.6625,NaN,S,315154,NONE
4,3,Hirvonen Mrs Alexander Helga E Lindqvist,female,22.0,1,1,12.2875,NaN,S,3101298,NONE


## Applying same steps for the test data

In [246]:
dummies = pd.get_dummies(test_data["Sex"])
test_data = pd.concat([test_data, dummies], axis=1)
test_data.drop(["Sex"], axis=1, inplace=True)

dummies = pd.get_dummies(test_data["Embarked"])
test_data = pd.concat([test_data, dummies], axis=1)
test_data.drop("Embarked", axis=1, inplace=True)

test_data["male"] = test_data["male"].astype(int)
test_data["female"] = test_data["female"].astype(int)
test_data["C"] = test_data["C"].astype(int)
test_data["Q"] = test_data["Q"].astype(int)
test_data["S"] = test_data["S"].astype(int)

def count_cabins(cabin):
    if pd.isna(cabin):  
        return 0
    else:
        return cabin.count(' ') + 1  
    
test_data['num_cabins'] = test_data['Cabin'].apply(count_cabins)

def has_ticket_item(item):
    if item == "NONE":
        return 0
    else:
        return 1
    
test_data['has_ticket_item'] = test_data['Ticket_item'].apply(has_ticket_item)


In [247]:
test_data = test_data[["Name", "Pclass", "female", "male", "Age", "SibSp", "Parch", "Fare", "num_cabins", "C", "Q", "S", "Ticket_number", "has_ticket_item"]]
test_data.head()

,Name,Pclass,female,male,Age,SibSp,Parch,Fare,num_cabins,C,Q,S,Ticket_number,has_ticket_item
0,Kelly Mr James,3,0,1,34.5,0,0,7.8292,0,0,1,0,330911,0
1,Wilkes Mrs James Ellen Needs,3,1,0,47.0,1,0,7.0000,0,0,0,1,363272,0
2,Myles Mr Thomas Francis,2,0,1,62.0,0,0,9.6875,0,0,1,0,240276,0
3,Wirz Mr Albert,3,0,1,27.0,0,0,8.6625,0,0,0,1,315154,0
4,Hirvonen Mrs Alexander Helga E Lindqvist,3,1,0,22.0,1,1,12.2875,0,0,0,1,3101298,0


# Results

In [248]:
train_data.head()

,Name,Pclass,female,male,Age,SibSp,Parch,Fare,num_cabins,C,Q,S,Ticket_number,has_ticket_item
0,Braund Mr Owen Harris,3,0,1,22.0,1,0,7.2500,0,0,0,1,21171,1
1,Cumings Mrs John Bradley Florence Briggs Thayer,1,1,0,38.0,1,0,71.2833,1,1,0,0,17599,1
2,Heikkinen Miss Laina,3,1,0,26.0,0,0,7.9250,0,0,0,1,3101282,1
3,Futrelle Mrs Jacques Heath Lily May Peel,1,1,0,35.0,1,0,53.1000,1,0,0,1,113803,0
4,Allen Mr William Henry,3,0,1,35.0,0,0,8.0500,0,0,0,1,373450,0


In [249]:
test_data.head()

,Name,Pclass,female,male,Age,SibSp,Parch,Fare,num_cabins,C,Q,S,Ticket_number,has_ticket_item
0,Kelly Mr James,3,0,1,34.5,0,0,7.8292,0,0,1,0,330911,0
1,Wilkes Mrs James Ellen Needs,3,1,0,47.0,1,0,7.0000,0,0,0,1,363272,0
2,Myles Mr Thomas Francis,2,0,1,62.0,0,0,9.6875,0,0,1,0,240276,0
3,Wirz Mr Albert,3,0,1,27.0,0,0,8.6625,0,0,0,1,315154,0
4,Hirvonen Mrs Alexander Helga E Lindqvist,3,1,0,22.0,1,1,12.2875,0,0,0,1,3101298,0


In [250]:
train_data_survived = processed_train_df["Survived"]
train_data_survived.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

# Now, we can create the model